# Notebook 07: Évaluation du Système et Logging avec Weights & Biases

Ce notebook montre comment utiliser nos modules d'évaluation (`RagEvaluator`, `SynthesisEvaluator`) pour mesurer la performance de "Cognitive Swarm" et comment intégrer ces résultats avec Weights & Biases (W&B) en utilisant `WandBMetricsLogger`.

**Prérequis :**
* **Environnement de Base :** Avoir exécuté le notebook `00_setup_environment.ipynb` pour configurer l'environnement Conda, les dépendances Python (notamment `wandb`, `pandas`, et les bibliothèques LangChain/LlamaIndex nécessaires), et s'assurer que le fichier `.env` à la racine du projet est correctement rempli.
* **Configuration des Fournisseurs (dans `.env`) :** Les évaluateurs de ce notebook dépendent des configurations suivantes :
    * **Pour `RagEvaluator` (via `RetrievalEngine`) :** La configuration de `DEFAULT_EMBEDDING_PROVIDER` et les variables associées (clés API, URLs, noms de modèles) sont cruciales pour l'embedding des requêtes.
        * Si `DEFAULT_EMBEDDING_PROVIDER` est `"openai"` : `OPENAI_API_KEY`, `OPENAI_EMBEDDING_MODEL_NAME`.
        * Si `DEFAULT_EMBEDDING_PROVIDER` est `"huggingface"` (local Sentence Transformers) : `HUGGINGFACE_EMBEDDING_MODEL_NAME`.
        * Si `DEFAULT_EMBEDDING_PROVIDER` est `"ollama"` : `OLLAMA_BASE_URL`, `OLLAMA_EMBEDDING_MODEL_NAME` et le modèle Ollama correspondant doit être disponible localement.
    * **Pour `SynthesisEvaluator` (LLM Juge via `get_llm`) :** La configuration de `DEFAULT_LLM_MODEL_PROVIDER` et les variables associées (clés API, URLs, noms de modèles) sont utilisées.
        * Si `DEFAULT_LLM_MODEL_PROVIDER` est `"openai"` : `OPENAI_API_KEY`, `DEFAULT_OPENAI_GENERATIVE_MODEL`.
        * Si `DEFAULT_LLM_MODEL_PROVIDER` est `"huggingface_api"` : `HUGGINGFACE_API_KEY`, `HUGGINGFACE_REPO_ID`.
        * Si `DEFAULT_LLM_MODEL_PROVIDER` est `"ollama"` : `OLLAMA_BASE_URL`, `OLLAMA_GENERATIVE_MODEL_NAME` et le modèle Ollama correspondant doit être disponible localement.
* **Base de Données MongoDB :** `MONGODB_URI` doit être configuré dans `.env` et l'instance accessible. Pour que l'évaluation RAG soit significative, la base de données doit être peuplée (via `01_data_ingestion_and_embedding.ipynb` ou `scripts/run_ingestion.py`) avec des embeddings correspondant au `DEFAULT_EMBEDDING_PROVIDER` actif.
* **Jeux de Données d'Évaluation :**
    * Pour `RagEvaluator` : Un fichier JSON (par exemple, `data/evaluation/rag_eval_dataset.json`, chemin configurable via `settings.EVALUATION_DATASET_PATH`) contenant des requêtes et les `chunk_id` pertinents attendus. Un jeu de données de démo interne est utilisé si le fichier n'est pas trouvé ou spécifié.
    * Pour `SynthesisEvaluator` : Des triplets `(requête, contexte, synthèse_à_évaluer)` sont nécessaires. Ce notebook utilise des exemples directement dans le code, mais pour une évaluation à plus grande échelle, un fichier de données serait utilisé (comme dans `scripts/run_evaluation.py`).
* **Weights & Biases :** Pour logger les métriques sur W&B, `WANDB_API_KEY` doit être configurée dans `.env` (ou vous devez être connecté via `wandb login`). Le logging W&B peut être désactivé.

In [ ]:
import logging
import sys
from pathlib import Path
import os
import json
import asyncio
import pandas as pd # Pour créer des tables pour W&B
from typing import Optional, List, Dict, Any 

# --- Gestion asyncio pour Jupyter ---
import nest_asyncio
nest_asyncio.apply()
# --- Fin Gestion asyncio ---

# --- Configuration du PYTHONPATH et chargement de .env ---
# NOTE IMPORTANTE SUR LE CWD (Current Working Directory) :
# La ligne suivante `project_root = Path().resolve().parent` suppose que le CWD du notebook
# est le dossier `/notebooks/`. Si vous avez configuré VS Code pour que le CWD
# soit la racine du projet (`cognitive-swarm-agents/`), alors `Path().resolve()`
# donnerait déjà la racine du projet, et vous devriez utiliser :
# project_root = Path().resolve()
# Vérifiez votre CWD avec `import os; print(os.getcwd())` ou `from pathlib import Path; print(Path().resolve())` pour confirmer.
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    print(f"Ajout de {project_root} au PYTHONPATH")

from dotenv import load_dotenv
# De même, si CWD est la racine du projet, dotenv_path serait `Path().resolve() / ".env"`
dotenv_path = project_root / ".env"
if dotenv_path.exists():
    load_dotenv(dotenv_path=dotenv_path)
    print(f"Variables d'environnement chargées depuis : {dotenv_path}")
else:
    print(f"ATTENTION: Fichier .env non trouvé à {dotenv_path}. Assurez-vous qu'il est à la racine du projet.")

from config.settings import settings
from config.logging_config import setup_logging

from src.rag.retrieval_engine import RetrievalEngine 
from src.evaluation.rag_evaluator import RagEvaluator, RagEvaluationMetrics
from src.evaluation.synthesis_evaluator import SynthesisEvaluator, SynthesisEvaluationResult, EvaluationAspectScore
from src.evaluation.metrics_logger import WandBMetricsLogger
from src.vector_store.mongodb_manager import MongoDBManager 

setup_logging(level="INFO")
logger = logging.getLogger("nb_07_evaluation_logging")

# --- Vérifications critiques pour ce notebook ---
logger.info(f"--- Configuration Active pour l'Évaluation (depuis settings.py et .env) ---")
config_valid = True # Indicateur global de validité de la configuration

# 1. Fournisseur d'embedding (pour RagEvaluator via RetrievalEngine)
active_embedding_provider = settings.DEFAULT_EMBEDDING_PROVIDER.lower()
logger.info(f"Fournisseur d'embedding (pour RAG) : '{active_embedding_provider}'")
if active_embedding_provider == "openai":
    if not (settings.OPENAI_API_KEY and settings.OPENAI_EMBEDDING_MODEL_NAME):
        logger.error("ERREUR : Embedding Provider est 'openai', mais OPENAI_API_KEY et/ou OPENAI_EMBEDDING_MODEL_NAME manquants.")
        config_valid = False
elif active_embedding_provider == "huggingface":
    if not settings.HUGGINGFACE_EMBEDDING_MODEL_NAME:
        logger.error("ERREUR : Embedding Provider est 'huggingface', mais HUGGINGFACE_EMBEDDING_MODEL_NAME manquant.")
        config_valid = False
elif active_embedding_provider == "ollama":
    if not (settings.OLLAMA_BASE_URL and settings.OLLAMA_EMBEDDING_MODEL_NAME):
        logger.error("ERREUR : Embedding Provider est 'ollama', mais OLLAMA_BASE_URL et/ou OLLAMA_EMBEDDING_MODEL_NAME manquants.")
        config_valid = False
else:
    logger.error(f"ERREUR : Fournisseur d'embedding inconnu : '{active_embedding_provider}'")
    config_valid = False

# 2. Fournisseur LLM Juge (pour SynthesisEvaluator via get_llm)
llm_judge_provider = settings.DEFAULT_LLM_MODEL_PROVIDER.lower()
logger.info(f"Fournisseur LLM Juge (pour Évaluation de Synthèse) : '{llm_judge_provider}'")
if llm_judge_provider == "openai":
    if not (settings.OPENAI_API_KEY and settings.DEFAULT_OPENAI_GENERATIVE_MODEL):
        logger.error("ERREUR : LLM Juge Provider est 'openai', mais OPENAI_API_KEY et/ou DEFAULT_OPENAI_GENERATIVE_MODEL manquants.")
        config_valid = False
elif llm_judge_provider == "huggingface_api":
    if not (settings.HUGGINGFACE_API_KEY and settings.HUGGINGFACE_REPO_ID):
        logger.error("ERREUR : LLM Juge Provider est 'huggingface_api', mais HUGGINGFACE_API_KEY et/ou HUGGINGFACE_REPO_ID manquants.")
        config_valid = False
elif llm_judge_provider == "ollama":
    if not (settings.OLLAMA_BASE_URL and settings.OLLAMA_GENERATIVE_MODEL_NAME):
        logger.error("ERREUR : LLM Juge Provider est 'ollama', mais OLLAMA_BASE_URL et/ou OLLAMA_GENERATIVE_MODEL_NAME manquants.")
        config_valid = False
else:
    logger.error(f"ERREUR : Fournisseur LLM Juge inconnu : '{llm_judge_provider}'")
    config_valid = False

# 3. Weights & Biases
if not settings.WANDB_API_KEY and not os.environ.get("WANDB_API_KEY"): 
    logger.warning("WANDB_API_KEY non trouvé. Le logging sur W&B pourrait échouer ou demander une authentification interactive.")
else:
    logger.info("Clé WANDB_API_KEY trouvée ou W&B est potentiellement configuré globalement.")

# 4. MongoDB
if not settings.MONGODB_URI or ("<user>" in settings.MONGODB_URI and "<password>" in settings.MONGODB_URI) or "<cluster_url>" in settings.MONGODB_URI:
    logger.error("ERREUR CRITIQUE : MONGODB_URI non trouvé ou semble non configuré. RetrievalEngine ne pourra pas fonctionner pour l'évaluation RAG.")
    config_valid = False
else:
    logger.info("MongoDB URI configuré.")

if not config_valid:
    logger.critical("Des configurations essentielles sont manquantes ou incorrectes. Veuillez vérifier votre fichier .env et settings.py. Les évaluations risquent d'échouer.")

# Nom de la collection pour les tests RAG
USE_NOTEBOOK_TEST_COLLECTION_FOR_RAG_EVAL = True 
if USE_NOTEBOOK_TEST_COLLECTION_FOR_RAG_EVAL:
    COLLECTION_NAME_EVAL = "arxiv_chunks_notebook_test" 
    logger.info(f"Évaluation RAG ciblera la collection de test du notebook: '{COLLECTION_NAME_EVAL}'")
else:
    COLLECTION_NAME_EVAL = MongoDBManager.DEFAULT_CHUNK_COLLECTION_NAME
    logger.info(f"Évaluation RAG ciblera la collection principale par défaut: '{COLLECTION_NAME_EVAL}'")

logger.info("--- Fin de la Vérification de Configuration Active ---")

### 1. Connexion à Weights & Biases

Avant de commencer les évaluations, nous allons nous assurer que nous pouvons nous connecter à W&B. Le `WandBMetricsLogger` s'en chargera. Si vous n'êtes pas déjà connecté via le CLI (`wandb login`), la présence de `WANDB_API_KEY` dans votre `.env` est fortement recommandée.

In [ ]:
# Cell from 07_evaluation_and_logging.ipynb (typically the one after W&B setup)
# Make sure to replace the entire content of this cell in your notebook.

# La variable COLLECTION_NAME_EVAL est définie dans la cellule précédente (ID b67db443)

def run_rag_evaluation_notebook_demo(top_k_eval: int = 3) -> Optional[RagEvaluationMetrics]: 
    logger.info(f"--- Début de l'Évaluation RAG (k={top_k_eval}) ---")
    
    can_proceed = True
    if not settings.MONGODB_URI or ("<user>" in settings.MONGODB_URI): 
        logger.error("RAG EVAL ERROR: MONGODB_URI non configuré correctement pour RetrievalEngine.")
        can_proceed = False

    active_embedding_provider_check = settings.DEFAULT_EMBEDDING_PROVIDER.lower()
    if active_embedding_provider_check == "openai":
        if not (settings.OPENAI_API_KEY and settings.OPENAI_EMBEDDING_MODEL_NAME):
            logger.error("RAG EVAL ERROR: Configuration OpenAI embedding incomplète (OPENAI_API_KEY ou OPENAI_EMBEDDING_MODEL_NAME).")
            can_proceed = False
    elif active_embedding_provider_check == "huggingface": 
        if not settings.HUGGINGFACE_EMBEDDING_MODEL_NAME:
            logger.error("RAG EVAL ERROR: HUGGINGFACE_EMBEDDING_MODEL_NAME non configuré pour le provider 'huggingface'.")
            can_proceed = False
    elif active_embedding_provider_check == "ollama":
        if not (settings.OLLAMA_BASE_URL and settings.OLLAMA_EMBEDDING_MODEL_NAME):
            logger.error("RAG EVAL ERROR: Configuration Ollama embedding incomplète (OLLAMA_BASE_URL ou OLLAMA_EMBEDDING_MODEL_NAME).")
            can_proceed = False
    else: 
        logger.error(f"RAG EVAL ERROR: Fournisseur d'embedding '{active_embedding_provider_check}' non supporté ou mal configuré.")
        can_proceed = False

    if not can_proceed:
        print("ERREUR: Prérequis pour l'évaluation RAG non remplis. Vérifiez les logs et votre configuration .env.")
        return None

    config_for_wandb_rag = {
        "evaluation_type": "RAG_Notebook_Demo",
        "retrieval_top_k_eval": top_k_eval,
        "rag_dataset_path": str(settings.EVALUATION_DATASET_PATH) if settings.EVALUATION_DATASET_PATH else "Config: Internal Default or Not Specified",
        "mongo_collection": COLLECTION_NAME_EVAL, 
        "active_embedding_provider": active_embedding_provider_check
    }
    if active_embedding_provider_check == "openai":
        config_for_wandb_rag["active_embedding_model"] = settings.OPENAI_EMBEDDING_MODEL_NAME
        config_for_wandb_rag["active_embedding_dimension"] = settings.OPENAI_EMBEDDING_DIMENSION
    elif active_embedding_provider_check == "huggingface":
        config_for_wandb_rag["active_embedding_model"] = settings.HUGGINGFACE_EMBEDDING_MODEL_NAME
        config_for_wandb_rag["active_embedding_dimension"] = settings.HUGGINGFACE_EMBEDDING_MODEL_DIMENSION
    elif active_embedding_provider_check == "ollama":
        config_for_wandb_rag["active_embedding_model"] = settings.OLLAMA_EMBEDDING_MODEL_NAME
        config_for_wandb_rag["active_embedding_dimension"] = settings.OLLAMA_EMBEDDING_MODEL_DIMENSION

    wb_logger = WandBMetricsLogger(
        project_name="CognitiveSwarm-Evaluations-Notebook", 
        run_name=f"RAG_Eval_Demo_k{top_k_eval}_{active_embedding_provider_check.replace('/', '_')}",
        config_to_log=config_for_wandb_rag,
        tags=["rag", "notebook_demo", f"k{top_k_eval}", active_embedding_provider_check]
    )

    rag_metrics_result: Optional[RagEvaluationMetrics] = None
    run_started_by_this_func = False 

    try:
        if not wb_logger.is_disabled:
            if wb_logger.start_run(): 
                 run_started_by_this_func = True
            else: 
                logger.warning("W&B run n'a pas pu démarrer via wb_logger.start_run(). Le logging W&B sera désactivé.")
        
        retrieval_engine_instance_rag = RetrievalEngine(collection_name=COLLECTION_NAME_EVAL)
        
        rag_eval_dataset_path_input = Path(settings.EVALUATION_DATASET_PATH) if settings.EVALUATION_DATASET_PATH and Path(settings.EVALUATION_DATASET_PATH).exists() else None
        evaluator_rag = RagEvaluator(
            retrieval_engine=retrieval_engine_instance_rag,
            eval_dataset_path=rag_eval_dataset_path_input
        )

        if run_started_by_this_func and wb_logger.wandb_run:
            # MODIFIED LINE: Using the new attribute name 'dataset_source_path'
            actual_dataset_path_logged = str(evaluator_rag.dataset_source_path) if evaluator_rag.dataset_source_path else "Internal Default Demo"
            if config_for_wandb_rag.get("rag_dataset_path") != actual_dataset_path_logged:
                 wb_logger.log_configuration({"rag_dataset_path_used": actual_dataset_path_logged})

        if not evaluator_rag.eval_dataset:
            logger.error("Jeu de données d'évaluation RAG vide ou non chargé. Arrêt de l'évaluation RAG.")
        else:
            # MODIFIED LINE: Using the new attribute name 'dataset_source_path' for logging
            logger.info(f"Évaluation RAG avec {len(evaluator_rag.eval_dataset)} requêtes (Dataset: {str(evaluator_rag.dataset_source_path) if evaluator_rag.dataset_source_path else 'Internal Default Demo'}).")
            rag_metrics_result = evaluator_rag.evaluate(k=top_k_eval)

            if rag_metrics_result:
                evaluator_rag.print_results(rag_metrics_result)
                if run_started_by_this_func and wb_logger.wandb_run:
                    wb_logger.log_rag_evaluation_results(rag_metrics_result, eval_name=f"RAG_Demo_Eval_k{top_k_eval}")
                    
                    eval_details_list = []
                    for item in evaluator_rag.eval_dataset: 
                        eval_details_list.append({
                            "query_id": item["query_id"],
                            "query_text": item["query_text"],
                            "expected_relevant_chunk_ids": ", ".join(item["expected_relevant_chunk_ids"])
                        })
                    if eval_details_list:
                        try:
                            details_df = pd.DataFrame(eval_details_list)
                            wb_logger.log_dataframe_as_table(details_df, "RAG_Evaluation_Queries_Used_Demo")
                        except Exception as e_df:
                            logger.error(f"Erreur lors du logging de la table de détails RAG : {e_df}")
            else:
                logger.warning("L'évaluation RAG n'a pas produit de métriques.")
                
    except Exception as e:
        logger.error(f"Une exception s'est produite pendant l'évaluation RAG : {e}", exc_info=True)
        if run_started_by_this_func and wb_logger.wandb_run: 
            try:
                wb_logger.log_summary_metrics({"rag_eval_error": 1, "error_message": str(e)[:250]})
            except Exception as e_log:
                logger.error(f"Erreur lors du logging de l'erreur RAG sur W&B : {e_log}")
    finally:
        if run_started_by_this_func and wb_logger.wandb_run: 
            wb_logger.end_run()
            logger.info("Run W&B pour l'évaluation RAG terminé.")
            
    return rag_metrics_result

# --- Exécution de la démo d'évaluation RAG ---
print("Lancement de la démonstration d'évaluation RAG...") # Already in user's code
rag_evaluation_results = run_rag_evaluation_notebook_demo(top_k_eval=3) # Already in user's code

if rag_evaluation_results:
    print("\n✅ Évaluation RAG terminée avec succès et métriques produites.") # Already in user's code
else:
    print("\n⚠️ L'évaluation RAG a échoué, n'a pas pu démarrer en raison de prérequis manquants, ou n'a pas produit de métriques. Vérifiez les logs ci-dessus.") # Already in user's code

### 2. Démonstration de l'Évaluation RAG

Nous allons instancier `RetrievalEngine` et `RagEvaluator`, exécuter une évaluation sur un petit jeu de données (le jeu par défaut de `RagEvaluator` ou un fichier JSON si configuré), puis logger les résultats et la configuration sur W&B.

**Action Requise :** Pour des résultats significatifs, assurez-vous d'avoir un fichier `rag_eval_dataset.json` (chemin configurable via `settings.EVALUATION_DATASET_PATH`) contenant des requêtes et les `chunk_id` pertinents attendus pour votre corpus. Le `RagEvaluator` utilisera son jeu de données de démo interne si ce fichier n'est pas trouvé ou non spécifié.

In [ ]:
# La variable LOG_LEVEL_NOTEBOOK est définie dans la première cellule (ID b67db443)
# nest_asyncio.apply() a aussi été appelé dans la première cellule.

async def run_synthesis_evaluation_notebook_demo(
    judge_model_name_override: Optional[str] = None
) -> Optional[SynthesisEvaluationResult]: 
    
    actual_judge_provider = settings.DEFAULT_LLM_MODEL_PROVIDER.lower()
    actual_judge_model_name = judge_model_name_override 
    
    if not actual_judge_model_name:
        if actual_judge_provider == "openai":
            actual_judge_model_name = settings.DEFAULT_OPENAI_GENERATIVE_MODEL
        elif actual_judge_provider == "huggingface_api":
            actual_judge_model_name = settings.HUGGINGFACE_REPO_ID
        elif actual_judge_provider == "ollama":
            actual_judge_model_name = settings.OLLAMA_GENERATIVE_MODEL_NAME
        else:
            actual_judge_model_name = "Unknown_Default_Model"
            
    logger.info(f"--- Début de l'Évaluation de la Synthèse (Juge LLM Provider: {actual_judge_provider}, Juge LLM Model: {actual_judge_model_name}) ---")

    can_proceed = True
    if actual_judge_provider == "openai":
        if not (settings.OPENAI_API_KEY and settings.DEFAULT_OPENAI_GENERATIVE_MODEL):
            logger.error("SYNTHESIS EVAL ERROR: Configuration OpenAI incomplète pour le LLM juge (OPENAI_API_KEY ou nom du modèle).")
            can_proceed = False
    elif actual_judge_provider == "huggingface_api":
        if not (settings.HUGGINGFACE_API_KEY and settings.HUGGINGFACE_REPO_ID): 
            logger.error("SYNTHESIS EVAL ERROR: Configuration HuggingFace API incomplète pour le LLM juge (HUGGINGFACE_API_KEY ou HUGGINGFACE_REPO_ID).")
            can_proceed = False
    elif actual_judge_provider == "ollama":
        if not (settings.OLLAMA_BASE_URL and settings.OLLAMA_GENERATIVE_MODEL_NAME): 
            logger.error("SYNTHESIS EVAL ERROR: Configuration Ollama incomplète pour le LLM juge (OLLAMA_BASE_URL ou OLLAMA_GENERATIVE_MODEL_NAME).")
            can_proceed = False
    else:
        logger.error(f"SYNTHESIS EVAL ERROR: Fournisseur LLM Juge inconnu ou non supporté: '{actual_judge_provider}'")
        can_proceed = False
        
    if not can_proceed:
        print(f"ERREUR: Prérequis pour le LLM Juge (provider: {actual_judge_provider}) non remplis. Vérifiez les logs et votre configuration .env.")
        return None

    sample_query_synth = "What are the main challenges in robotic grasping using reinforcement learning?"
    sample_context_synth = """
Challenge 1: Sample Inefficiency. RL algorithms often require a vast amount of data (trials) to learn effective grasping policies. This is costly and time-consuming on physical robots.
Challenge 2: Sim-to-Real Gap. Models trained in simulation may not transfer well to real robots due to differences in dynamics, sensing, and appearance.
Challenge 3: Reward Design. Crafting appropriate reward functions that guide the agent towards successful and robust grasping without unintended behaviors is difficult.
Challenge 4: High-Dimensional State/Action Spaces. Grasping involves continuous and high-dimensional inputs (e.g., camera images) and outputs (e.g., robot joint commands).
(Source: Fictional summary based on general knowledge for demo purposes)
"""
    sample_synthesis_to_eval = """
Robotic grasping using reinforcement learning faces several key challenges. Firstly, sample inefficiency means many trials are needed.
Secondly, bridging the sim-to-real gap is problematic due to mismatches between simulation and reality.
Thirdly, designing effective reward functions is complex. Lastly, the high dimensionality of state and action spaces poses difficulties.
These challenges are actively being researched.
"""

    config_for_wandb_synth = {
        "evaluation_type": "Synthesis_Notebook_Demo",
        "original_query_snippet": sample_query_synth[:100] + "..." if len(sample_query_synth) > 100 else sample_query_synth,
        "judge_llm_provider": actual_judge_provider,
        "judge_llm_model_used": actual_judge_model_name
    }

    wb_logger_synth = WandBMetricsLogger(
        project_name="CognitiveSwarm-Evaluations-Notebook",
        run_name=f"Synth_Eval_Demo_Judge_{actual_judge_provider}_{str(actual_judge_model_name).split('/')[-1].replace('.', '_')}",
        config_to_log=config_for_wandb_synth,
        tags=["synthesis", "notebook_demo", "llm_as_judge", actual_judge_provider]
    )
    
    synthesis_metrics_result: Optional[SynthesisEvaluationResult] = None
    run_started_by_this_func = False

    try:
        if not wb_logger_synth.is_disabled:
            if wb_logger_synth.start_run():
                run_started_by_this_func = True
            else:
                logger.warning("W&B run n'a pas pu démarrer pour l'évaluation de synthèse.")
        
        evaluator_synth = SynthesisEvaluator(
            judge_llm_provider=None, 
            judge_llm_model_name=judge_model_name_override 
        )
        
        logger.info(f"Évaluation de la synthèse pour la requête: '{sample_query_synth[:50]}...'")
        synthesis_metrics_result = await evaluator_synth.evaluate_synthesis(
            query=sample_query_synth,
            synthesis=sample_synthesis_to_eval,
            context=sample_context_synth
        )

        if synthesis_metrics_result:
            evaluator_synth.print_results(synthesis_metrics_result, query=sample_query_synth)
            if run_started_by_this_func and wb_logger_synth.wandb_run:
                wb_logger_synth.log_synthesis_evaluation_results(synthesis_metrics_result, eval_name="Synthesis_Demo_Eval")
                
                synth_detail_data = [{
                    "query": sample_query_synth,
                    "context_snippet": sample_context_synth[:500] + "..." if len(sample_context_synth) > 500 else sample_context_synth,
                    "evaluated_synthesis": sample_synthesis_to_eval,
                    "relevance_score": synthesis_metrics_result.get("relevance", {}).get("score") if synthesis_metrics_result.get("relevance") else None,
                    "relevance_reasoning": synthesis_metrics_result.get("relevance", {}).get("reasoning") if synthesis_metrics_result.get("relevance") else None,
                    "faithfulness_score": synthesis_metrics_result.get("faithfulness", {}).get("score") if synthesis_metrics_result.get("faithfulness") else None,
                    "faithfulness_reasoning": synthesis_metrics_result.get("faithfulness", {}).get("reasoning") if synthesis_metrics_result.get("faithfulness") else None,
                }]
                try:
                    details_df_synth = pd.DataFrame(synth_detail_data)
                    wb_logger_synth.log_dataframe_as_table(details_df_synth, "Synthesis_Evaluation_Run_Detail_Demo")
                except Exception as e_df_s:
                    logger.error(f"Erreur lors du logging de la table de détails de synthèse: {e_df_s}")
        else:
            logger.warning("L'évaluation de la synthèse n'a pas produit de métriques.")

    except ValueError as ve: 
        logger.error(f"Erreur de configuration LLM pour SynthesisEvaluator: {ve}", exc_info=True)
        if run_started_by_this_func and wb_logger_synth.wandb_run:
             wb_logger_synth.log_summary_metrics({"synthesis_eval_error": 1, "error_message": str(ve)[:250]})
    except Exception as e:
        logger.error(f"Une exception s'est produite pendant l'évaluation de la synthèse : {e}", exc_info=True)
        if run_started_by_this_func and wb_logger_synth.wandb_run:
            wb_logger_synth.log_summary_metrics({"synthesis_eval_error": 1, "error_message": str(e)[:250]})
    finally:
        if run_started_by_this_func and wb_logger_synth.wandb_run:
            wb_logger_synth.end_run()
            logger.info("Run W&B pour l'évaluation de synthèse terminé.")
            
    return synthesis_metrics_result

# --- Exécution de la démo d'évaluation de synthèse ---
print("Lancement de la démonstration d'évaluation de la synthèse...")
synthesis_evaluation_results = asyncio.run(run_synthesis_evaluation_notebook_demo(judge_model_name_override=None))

if synthesis_evaluation_results:
    print("\n✅ Évaluation de la synthèse terminée avec succès et métriques produites.")
else:
    print("\n⚠️ L'évaluation de la synthèse a échoué, n'a pas pu démarrer, ou n'a pas produit de métriques. Vérifiez les logs ci-dessus.")

### 3. Démonstration de l'Évaluation de la Synthèse

Nous allons maintenant évaluer la qualité d'une synthèse (pertinence, fidélité) en utilisant `SynthesisEvaluator` et un LLM comme juge.

**Action Requise :** Pour cette section, vous devez fournir :
1.  Une **requête utilisateur** (`sample_query_synth`).
2.  Un **contexte** (`sample_context_synth`) qui aurait été fourni à l'agent de synthèse pour générer la réponse.
3.  Une **synthèse à évaluer** (`sample_synthesis_to_eval`) qui est la sortie de votre `SynthesisAgent` pour cette requête et ce contexte.

Vous pouvez obtenir ces éléments en exécutant le notebook `04_langgraph_workflow_design.ipynb` (ou `scripts/run_cognitive_swarm.py`) pour une requête donnée et en copiant/collant la requête, le contexte (qui peut être une combinaison des messages pertinents ou des chunks récupérés) et la sortie de synthèse finale.

### 4. Utilisation du Script `scripts/run_evaluation.py`

Pour des évaluations plus complètes et automatisées, notamment sur des jeux de données plus larges, il est recommandé d'utiliser le script CLI `scripts/run_evaluation.py`.

Ce script orchestre les `RagEvaluator` et `SynthesisEvaluator`, gère le chargement des datasets depuis des fichiers, et intègre le logging W&B de manière configurable.

**Exemple de commande (à exécuter dans votre terminal, depuis la racine du projet) :**
```bash
python -m scripts.run_evaluation --eval_type all \
    --rag_dataset data/evaluation/rag_eval_dataset.json \
    --synthesis_dataset data/evaluation/synthesis_eval_dataset.json \
    --wandb_project "CognitiveSwarm-MainEvals" \
    --wandb_run_name "Full_Eval_Run_$(date +%Y%m%d_%H%M)" \
    --wandb_tags "full_eval,scheduled" \
    --log_level INFO

## Conclusion de l'Évaluation et du Logging

Ce notebook a montré comment :
- Utiliser `RagEvaluator` pour évaluer la performance de récupération.
- Utiliser `SynthesisEvaluator` avec un LLM comme juge pour évaluer la qualité des synthèses.
- Intégrer ces évaluations avec `WandBMetricsLogger` pour un suivi sur Weights & Biases.

L'évaluation continue est une partie essentielle du développement de systèmes LLM robustes. Les métriques collectées peuvent guider les améliorations des prompts, des stratégies RAG, des modèles LLM choisis, etc.

N'oubliez pas de consulter votre tableau de bord Weights & Biases pour visualiser les résultats loggés !